In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import math

In [2]:
df = pd.read_csv("C:/Users/Elio Padilla/Desktop/SOYHENRY/MODULO_LABS/JUEGOS_ML_OP/ETL/stems_games_expandido.csv")

    EL PROCESO DE ANALISIS DE LA DATA FUENTE A EFECTOS DE LA IMPLEMENTACION DE UN MODELO DE PREDICCION DE ML CONTEMPLARA LOS SIGUIENTES CRITERIOS, LISTADOS EN SU RESPECTIVO ORDEN DE RELEVANCIA.
    1.- USO DE LA MAYOR CANTIDAD DE DATOS POSIBLES. 
        ESTO IMPLICA QUE AQUELLAS COLUMNAS QUE DE ANTEMANO CONTRIBUYEN EN MUY POCA CANTIDAD DE DATOS, SERAN EXCLUIDAS.
    2.- MINIMIZACION DE LA CANTIDAD DE FEATURES PARA LA IMPLEMENTACION DE ML. ESTA PREMISA SE DESARROLLARA EN E SIGUIENTE ORDEN DE CRITERIOS.
        2.1.- ENTRE COLUMNAS CON CONTENIDO IGUAL O SIMILAR, PREVALECERA AQUELLA COLUMNA QUE APORTE MAYOR CANTIDAD DE INFORMACION.
        2.2.- ENTRE LOS FEATURES, AQUELLOS CUYA RELACION LINEAL SE CONSIDERE MUY ALTA, SE DECIDIRA POR UNO. AQUEL QUE APORTE MAYOR INFORMACION RELEVANTE.
    3.- SEGUN SEA EL MODELO CONSIDERADO, SE SELECCIONARAN LOS FEATURES QUE MEJOR SE DESEMPEÑEN.

In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'publisher', 'genres', 'tags', 'title', 'specs',
       'early_access', 'sentiment', 'metascore', 'año', 'precio_1'],
      dtype='object')

In [4]:
df = df.drop(['Unnamed: 0'], axis=1) # No aporta.
df = df.drop(["id"], axis = 1) # El dato id, no aporta nada al objetivo de la prediccion, po tanto sera desconsiderado desde este punto.
df = df.drop(["title"], axis = 1) # El titulo de un juego tampoco es considerado un elemento que dado el historico de nombres al juego, 
# defina impacto en una prediccion.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2088746 entries, 0 to 2088745
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   publisher     object 
 1   genres        object 
 2   tags          object 
 3   specs         object 
 4   early_access  bool   
 5   sentiment     object 
 6   metascore     float64
 7   año           int64  
 8   precio_1      float64
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 129.5+ MB


In [178]:
#df[df["precio_1"].isna()]

#df[df["genres"].isna()]

#df[df["specs"].isna()]

#df[df["año"] == -1] #Existen valores de año irregulares, pero no se restringira el registro 
# puesto que aun no se sabe si esta columna sera importante para el modelo.

#df[df["publisher"].isna()].head() # Solo hay 5 registros con valores nan, no se restringira 
# a menos que termine siendo considerada importante para el modelo.

#df[df["title"].isna()]

#df[df["early_access"].isna()]

#df[df["sentiment"].isna()].head()

#df[df["metascore"].notna()]["metascore"].count() #[332.330 es la cantidad de registros a este nivel con metascore diferente de nulo. 
# ante tan poca cantidad de registros con este valor, sera descontado desde ya del modelo.

#df[df["tags"].isna()] # la columna "tags" no sera utilizada en el modelo dado que contiene mucho mas datos nulos que genero, y estas dos columnas
# contienen aproximadamente el mismo conjunto de valores. Tal decision mantiene la premisa de mayor cant de datos coherentes.



In [6]:
#NO tiene sentido trabajar con registros en los cuales existan valores nulos.
df = df[df["precio_1"].notna()]
df = df[df["genres"].notna()]
df = df[df["specs"].notna()]
df = df[df["sentiment"].notna()]

Siendo "tags" una columna con valores muy similares a genero, razon por la cual laidea es avaluar el uso de una sola de ellas (genres o tags),
se ha identificado que el conjunto de campos tags con nulos es mayor que la misma situacion en generos. Razon por la cual se concluye a este nivel
del analisis que la columna "tags" no sera utilizada en el modelo ML, sera utilizada la columna genres.

In [7]:
df = df.drop(['tags'], axis=1)
df = df.drop(["metascore"], axis=1)

In [8]:
# Mostramos los valores del campo genero a los cuales esta asociado un valor irregular de precio (-1)
df[df["precio_1"] == -1][["genres","specs","precio_1"]]["genres"].unique()

array(['Free to Play', 'Indie', 'RPG', 'Strategy', 'Casual', 'Simulation',
       'Sports', 'Action', 'Massively Multiplayer', 'Adventure',
       'Early Access', 'Utilities', 'Animation &amp; Modeling',
       'Video Production', 'Design &amp; Illustration', 'Web Publishing',
       'Education', 'Software Training', 'Racing', 'Photo Editing'],
      dtype=object)

DE TODOS LOS GENEROS ASOCIADOS A VALORES IRREGULARES EN PRECIO, QUE PUDIERAN IMPLICAR UNA CONCLUSION ACEPTABLE, SON AQUELLOS "FREE TO PLAY"
PUES, SE ENTETIENDE QUE ESTA CUALIDAD IMPLICA PRECIO = 0. POR TANTO, SE PODRIA IMPUTAR CERO (0) A TODOS AQUELLOS CASOS EN LOS CUALES SE ENCONTRO UN
VALOR DE PRECIO IRREGULAR PERO SON "FREE TO PLAY"

In [9]:
# Pero cuantos valores "correctos", es decir, numericos en el precio, estan asociados al genero "free to play"
df_temp = df[df["genres"] == "Free to Play"]
df_temp[df_temp["precio_1"] == -1]["precio_1"].count()

38344

DADA LA INTENCION DE UTILIZAR LA MAYOR CANTIDAD DE DATOS POSIBLES, PERO TAMBIEN CON LA MAYOR COHERENCIA POSIBLE, SE HA IDENTIFICDO UNA SITUACION POTENCIALMENTE CRITICA. ESTOS ES, PRECIOS DIFERENTES DE CERO CUYO GENERO ES FREE TO PLAY. 

TAL SITUACION SE VUELVE CRITICA CUANDO TRATANDO DE ATENDER A LA POSIBLE IMPUTACION DE VALOR CERO A AQUELLOS PRECIOS IRREGULARES (IDENTIFICADOS CON VALOR ALFANUMERICO) SE NOS PLANTEA ENTONCES EL CONFLICTO CUANDO VERIFICAMOS QUE TAL PREMISA NO APLICA. EN ESE SENTIDO SURGE LA NECESIDD DE TOMAR UNA DECISION DE LAS SIGUIENTES DISPONIBLES:

1.- ASIGNAR A LA COLUMNA PRECIO CON VALORES IRREGULARES, EL VALOR CONTENIDO EN AQUELLOS REGISTROS CON VALOR NUMERICO EN PRECIO PERO DE GENERO FREE TO PLAY Y POTENCIALMENTE APLICAR DICHO CRITERIO SOBRE TODOS AQUELLOS PRECIOS IRREGULARES PERO ASOCIADO A VALORES DE GENERO.
2.- O, ASIGNAR EL VALOR CERO, A  AQUE LLOS PRECIOS "IRREGULARES" ASOCIADOS A GENERO FREE TO PLAY.

DECISION ASUMIDA, NO SE CONSIDERARAN A EFECTOS DEL MODELO AQUELLOS REGISTROS CUYO PRECIO SEA UN VALOR IRREGULAR. SE ASUMIRA QUE LOS REGISTROS RESTANTES ESTAN PROPORCIONANDO LA LEGITITIMIDAD NECESARIA DE LA INFORMACION.

In [10]:
df = df[df["precio_1"] != -1]

In [11]:
# No nos sirven registros repetidos puesto que el objetivo es abstraer datos que configures una prediccion al mismo tiempo que se minimice el dataset.
df = df.drop_duplicates()

In [12]:
df["precio_1"].count()

190768

FINALMENTE NOS QUEDAMOS CON UN DATASET EN ESTE PUNTO CON 190.768 REGISTROS.

In [13]:
df.head(10)

,publisher,genres,specs,early_access,sentiment,año,precio_1
740,Trickjump Games Ltd,Action,Single-player,False,Mixed,2018,3.99
741,Trickjump Games Ltd,Action,Steam Achievements,False,Mixed,2018,3.99
756,Trickjump Games Ltd,Adventure,Single-player,False,Mixed,2018,3.99
757,Trickjump Games Ltd,Adventure,Steam Achievements,False,Mixed,2018,3.99
772,Trickjump Games Ltd,Simulation,Single-player,False,Mixed,2018,3.99
773,Trickjump Games Ltd,Simulation,Steam Achievements,False,Mixed,2018,3.99
1457,Apillo,Adventure,Single-player,False,1 user reviews,2018,1.59
1458,Apillo,Adventure,Steam Achievements,False,1 user reviews,2018,1.59
1459,Apillo,Adventure,Steam Cloud,False,1 user reviews,2018,1.59
1478,Apillo,Casual,Single-player,False,1 user reviews,2018,1.59


ANALICEMOS A PARTIR DE AQUI A LAS VARIABLES DISPONIBLES EN EL SIGUIENTE ORDEN.
1.- COMPROTAMIENTO Y CARACTERISTICAS DE LA VARIABLE OBJETIVO O EXPLICADA "PRECIO"
2.- COMPORTAMIENTO Y CARACTERISTICAS DE LAS VARIABLES EXPLICADORAS. EN EL SIGUIENTE ORDEN DE RELEVANCIA (A PRIORI):
    2.1.- GENERO ("genres)
    2.2.- ESPECIFICACIONES ("specs")
    2.3.- SENTIMIENTO ("sentiment")
    2.4.- PUBLICADOR ("publisher")
    2.5.- ACCESO TEMPRANO O SALIDA TEMPRANA ("early_access")
    2.6.- AÑO ("año")


In [14]:
df.to_csv("DatasetML_V1.csv")